In [1]:
import numpy as np
import pandas as pd


file_path = r'C:\Users\jites\Desktop\Jupyter_practice\FInal_data_20_columns_2.xlsx'
df = pd.read_excel(file_path)

df = df.astype(str)
df = df.fillna("")
df = df[:2000]
df.head(2)

,ChiefComplaint,PresentProblemLocation,PreProbDuration,PreProbOnset,PreProbSeverity,AttackType,SmokingHabitType,PhysicalActivity,ALCHOLIC,DietryHabit,...,TreatmentType,SurgeryType,SurgeryDurationH,SurgeryDurationM,AnesthicTypeName,AnesRecoverySmoothType,RecorveryType,Followupdate,DoctorRemark,Paragraph
0,Ho intermittent pain abdomen and flatulence X ...,Abdomen problem,Complains from 1 month ago,Gradually it happen,nan,Intermittent attack,Patient does not have smoking habit,Light Worker,Does not drink alcohol,nan,...,Treatment type OPERATIVE,Surgery type CHOLECYSTECTOMY,nan,40 minutes,Anesthic type General,Anes smooth recovery,NORMAL recovery,Follow-up date 20 December 2014,nan,Ho intermittent pain abdomen and flatulence X ...
1,During routine health checkup gallstone detect...,Abdomen problem,Complains from 2 months ago,nan,Mild problem,No attack,Patient does not have smoking habit,Moderate Worker,Drinks alcohol,Non Vegetarian,...,Treatment type OPERATIVE,Surgery type CHOLECYSTECTOMY,nan,30 minutes,Anesthic type General,Anes smooth recovery,NORMAL recovery,Follow-up date 20 December 2014,nan,During routine health checkup gallstone detect...


In [2]:
from sklearn.model_selection import train_test_split

# Split the data into train and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [3]:
import torch

# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load pre-trained GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
# model = GPT2LMHeadModel.from_pretrained('./gpt2-finetuned-model-two-2').to(device)
# tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-finetuned-tokenizer-two-2')

tokenizer.pad_token = tokenizer.eos_token

# Regularization: Add dropout and weight decay
model.config.attn_pdrop = 0.2
model.config.embd_pdrop = 0.2
model.config.resid_pdrop = 0.2

In [5]:
# # Tokenize data
# def tokenize_function(examples):
#     return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

# # Create dataset from pandas DataFrame
# def dataset_from_dataframe(df, tokenizer):
#     texts = df['Paragraph'].tolist()
#     labels = [' '.join([f"{label}: {sentence}" for label, sentence in zip(df.columns[:-1], row[:-1])]) for row in df.itertuples(index=False)]
#     data = {'text': [f"{text} => {label}" for text, label in zip(texts, labels)]}
#     return TextDataset(
#         tokenizer=tokenizer,
#         file_path=None,
#         block_size=128,
#         overwrite_cache=True,
#         dataset=data,
#         cache_dir="./cached_dataset"
#     )
# train_dataset = dataset_from_dataframe(train_df, tokenizer)
# val_dataset = dataset_from_dataframe(val_df, tokenizer)



# # Function to create a temporary text file for TextDataset
# def dataset_from_dataframe(df, tokenizer, tmp_file_path):
#     texts = df['Paragraph'].tolist()
#     labels = [' '.join([f"{label}: {sentence}" for label, sentence in zip(df.columns[:-1], row[:-1])]) for row in df.itertuples(index=False)]

#     with open(tmp_file_path, 'w', encoding='utf-8') as f:
#         for text, label in zip(texts, labels):
#             f.write(f"{text} => {label}\n")

#     return TextDataset(
#         tokenizer=tokenizer,
#         file_path=tmp_file_path,
#         block_size=128,
#         overwrite_cache=True
#     )


# # File paths for temporary storage
# train_file_path = './train_data.txt'
# val_file_path = './val_data.txt'

# train_dataset = dataset_from_dataframe(train_df, tokenizer, train_file_path)
# val_dataset = dataset_from_dataframe(val_df, tokenizer, val_file_path)

# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=False
# )


In [6]:
from datasets import Dataset

def create_dataset(df):
    texts = df['Paragraph'].tolist()
    labels = [' '.join([f"{label}: {sentence}" for label, sentence in zip(df.columns[:-1], row[:-1])]) for row in df.itertuples(index=False)]
    
    # Create a list of dictionaries
    data = [{'text': f"{text} => {label}"} for text, label in zip(texts, labels)]
    return Dataset.from_list(data)

# Create Hugging Face datasets from DataFrames
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Initialize data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [7]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import EarlyStoppingCallback

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)       ## weight_decay =>  L2 regularization
scheduler = CosineAnnealingLR(optimizer, T_max=500, eta_min=5e-6)

In [8]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,      ## 2
    gradient_accumulation_steps=4,  # Accumulate over 4 batches to simulate a batch size of 32
    per_device_eval_batch_size=8,
    eval_steps=400,
    save_steps=500,
    warmup_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    # eval_strategy="steps",
    # save_strategy='steps',
    # weight_decay=0.01,  ## L2 regularization
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,  # Load best model at the end based on evaluation
    metric_for_best_model='eval_loss',
    greater_is_better=False,
)

# training_args = TrainingArguments(
#     output_dir=repo_name,
#     group_by_length=True,
#     length_column_name='input_length',
#     per_device_train_batch_size=24,
#     gradient_accumulation_steps=2,
#     evaluation_strategy="steps",
#     num_train_epochs=20,
#     fp16=True,
#     save_steps=1000,
#     save_strategy='steps', # we cannot set it to "no". Otherwise, the model cannot guess the best checkpoint.
#     eval_steps=1000,
#     logging_steps=1000,
#     learning_rate=5e-5,
#     warmup_steps=500,
#     save_total_limit=3,
#     load_best_model_at_end = True # this will let the model save the best checkpoint
# )

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    # train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    optimizers=(optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train model
trainer.train()

# Save model and tokenizer
model.save_pretrained('./gpt2-finetuned-model-two-4')
tokenizer.save_pretrained('./gpt2-finetuned-tokenizer-two-4')

  0%|          | 0/150 [00:00<?, ?it/s]

c:\Users\jites\anaconda3\envs\torchenv\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.27029281854629517, 'eval_runtime': 69.212, 'eval_samples_per_second': 5.779, 'eval_steps_per_second': 0.722, 'epoch': 1.0}
{'loss': 0.6584, 'grad_norm': 0.9943187236785889, 'learning_rate': 4.570288237343631e-05, 'epoch': 2.0}


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.2131483405828476, 'eval_runtime': 67.3374, 'eval_samples_per_second': 5.94, 'eval_steps_per_second': 0.743, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'train_runtime': 12734.9679, 'train_samples_per_second': 0.377, 'train_steps_per_second': 0.012, 'train_loss': 0.5115227381388346, 'epoch': 3.0}


('./gpt2-finetuned-tokenizer-two-4\\tokenizer_config.json',
 './gpt2-finetuned-tokenizer-two-4\\special_tokens_map.json',
 './gpt2-finetuned-tokenizer-two-4\\vocab.json',
 './gpt2-finetuned-tokenizer-two-4\\merges.txt',
 './gpt2-finetuned-tokenizer-two-4\\added_tokens.json')

In [9]:
# training_args = TrainingArguments(
#     output_dir='./results',
#     overwrite_output_dir=True,
#     num_train_epochs=5,
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
#     eval_steps=400,
#     save_steps=800,
#     warmup_steps=500,
#     eval_strategy="epoch",
#     logging_dir='./logs',
# )

# training_args = TrainingArguments(
#     overwrite_output_dir=True,
#     output_dir='./results',          # output directory
#     num_train_epochs=1,             # number of epochs
#     per_device_train_batch_size=2,  # batch size per device during training
#     per_device_eval_batch_size=2,   # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
#     eval_steps=200,
#     save_steps=500,
#     eval_strategy='epoch',     # evaluate every epoch
#     save_strategy='epoch',           # save model every epoch
#     load_best_model_at_end=True,     # load best model at end
#     metric_for_best_model='eval_loss', # metric to use to compare model
#     greater_is_better=False,         # eval_loss, so lower is better
#     learning_rate=1.005e-06
# )


# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )

# # Fine-tune the model
# trainer.train()

# # Save the model and tokenizer
# model.save_pretrained('./gpt2-finetuned-model-two-3')
# tokenizer.save_pretrained('./gpt2-finetuned-tokenizer-two-3')

In [13]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline
import time
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

start = time.time()

model = GPT2LMHeadModel.from_pretrained('./gpt2-finetuned-model-two-4').to(device)
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-finetuned-tokenizer-two-4')
# Set pad token to eos token
tokenizer.pad_token = tokenizer.eos_token

# Enable mixed precision if using a compatible GPU
model = model.half() if torch.cuda.is_available() else model

# Example of a generation function using a pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

def generate_labels(sent):
    paragraph = sent
    input_text = f"{paragraph} =>"

    # Encode with truncation and padding
    inputs = tokenizer.encode_plus(
        input_text,
        return_tensors='pt',
        max_length=512,  # Adjust based on your requirement
        truncation=True,  # Explicit truncation
        padding='max_length'  # Padding to the max length
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Generate predictions with optimized settings
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=1024,  # Adjust based on your requirement
            num_return_sequences=1,
            num_beams=2,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )

    # Process and return the output
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    labels = output_text.split('=>')[-1].strip()
    return labels

# Example usage
# paragraph = "ho intermittent pain abdomen and flatulence x 1 month . abdomen problem . complains from 1 month ago . gradually it happen . intermittent attack . patient does not have smoking habit . light worker . does not drink alcohol . past medical history non diabetic. non hypertensive. she underwent lscs in 2005 . cholelithiasis . treatment type operative . surgery type cholecystectomy . procedure follow cholecystectomy . 40 minutes . anesthic type general . anes smooth recovery . gb found to be well distended. gb was retracted and calot s area was disected. cystic duct artery clipped cut. gb disected out from the liver bed.it is extracted through the epigastric port. ports were closed. gb tissue sent for biopsy . normal recovery . follow-up date 20 december 2014 ."
paragraph = "Ho right hypochondrium discomfort since 5 6 months. Umbilical hernia detected in 20 December 2014. She is non diabetic and non hypertensive. She underwent LSCS 6 years ago. USG abdomen on 20 December 2014 revealed Umbilical hernia, benign hepatic cysts with pressure effect in RHD  fibroid. MDCT on 31 December 2014 showed hepatomegaly with asymmetrical fatty infiltration and multiple non enhancing cystic lesions in the right and caudate lobes showing regular peripheral rim without enhancing mural solid component. Suggest possibilities like biliary cyst adenomas  hepatic cysts. The largest lesion is compressing the right hepatic duct with prominence of corresponding biliary radicles. There are few prominent paraaortic and interaortocaval nodes. Gastric rugae are prominent. Visualized bowel loops show no significant wall thickening or dilatation. There is umbilical hernia. Patient again admitted on 03 January 2017 with history of right upper abdominal discomfort. No ho fever. She had past ho umbilical hernia repair and drainage of right simple liver cyst done in 2015. She is known hypertensive and on treatment. USG abdomen on 30 December 2016 revealed benign hepatic cyst  hydatid . Abdomen problem . Complains from 6 months ago . Gradually it happen . Mild problem . No attack . Patient does not have smoking habit . Light Worker . Does not drink alcohol . Both VegNon Veg . Prefer meals 2 3 times a day . Prefer oil mustrad oil . Past medical history LSCS 6 years ago . No medicine refered by other . Normal 0 MicrosoftInternetExplorer4  Style Definitions  table.MsoNormalTable mso style name Table Normal ; mso tstyle rowband size0; mso tstyle colband size0; mso style noshowyes; mso style parent""; mso padding alt0in 5.4pt 0in 5.4pt; mso para margin0in; mso para margin bottom.0001pt; mso paginationwidow orphan; font size10.0pt; font family Times New Roman ; 17 January 2015  Simple hepatic cyst segment Iv V with umbilical hernia omentocele 04 January 2017  Hepatic cyst rt liver segment VVI Hypertension . Treatment type OPERATIVE . Surgery type MULTIPLE . Procedure follow LAP DRAINAGE OF CYST OF CYST  LAP. ADHESIOLYSIS, RELEASE OF OMENTOCELE  MESH REPAIR OF HERNIA . Anesthic type General . Anes smooth recovery . Recommende drug INJ. PYROLATE 1 AMP .  INJ. PROPOFOL .  17 January 2015Lap. drainage of cyst lap. adhesiolysis, release of omentocele mesh repair of hernia  Lap. adhesiolysis of porta and hepatic bed. Caudate lobe Segment V VI dissected and lifting up. Deep seated cyst aspirated out. Hernia contents released using harmonic scalpel. 12x12 cm parietex mesh applied fixed with tacjer suture. 04 January 2017  Difficult lap. exploration hydatid cyst with marsupialization cholecystectomy   Cyst is situated in the postero inferior surface lateral to porta overlying upper pole of rt kidney Segment VVI. Cyst could not be exposed as GB was hindering the exploration. Lap. Cholecystectomy was done. Identification cyst was difficult as there was overlying liver tissue. Needle aspiration of cyst revealed golden bile stained fluid. Cyst was opened sucked out the scolices germinal layer. Cavity was treated with 3 NS 10 povidene iodine. Marsupialization of cyst was done. Active bleeding from cyst wall was controlled with harmonic scalpel. Margin was sutured. Operative field was irrigated with NS. The bile communication with the cyst could not be identified. Drainage applied within Sac ports were closed. Post operative biliary drainage was collected measured through drainage tube. Tissues sent for Biopsy . NORMAL recovery . Medicines suggested INJ. AMIKACIN 750 INJ. KETONOV 2AMP INJ. PANTOCID 40 . Follow-up date 28 January 2015 . CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS AVOID STRAIN . NO . "
# paragraph = "Severe pain abdomen 4 months ago . Abdomen problem . Suddenly it happen . Severe problem . Single attack . chronic calculus cholecystitis with chronic appendicitis . Treatment type OPERATIVE . Surgery type CHOLECYSTECTOMY + APPENDICECTOMY . Procedure follow CHOLECYSTECTOMY + APPENDICECTOMY . Anesthic type General . Anes smooth recovery . GB contracted multiple cal. CBD normal Appendix chronically inflammed . Follow-up date 29 April 2000 . "
# paragraph = "Co flatulence, intermitent pain 1 year. increases in empty stomach  after taking spicy food. Patient states that around the leter helf of 2017 she used to experience severe abdominal pain whichb used to radiate . Pain was sudden on onset radiating to the back, subsided gradually  would recur almost every 15 days . Abdomen problem . No attack . Patient does not have smoking habit . Light Worker . Does not drink alcohol . Both VegNon Veg . Prefer oil mustrad oil . Past medical history Not a known case of T2DMHTNThyroid. LSCS 1 yr back . Cholelithiasis . Treatment type OPERATIVE . Surgery type CHOLECYSTECTOMY . Procedure follow CHOLECYSTECTOMY . 22 minutes . Anesthic type General . Anes smooth recovery . Lap Chole performed under GA . NORMAL recovery . NO . "

print(paragraph)
print()
print()

result = generate_labels(paragraph)

print(result)
end = time.time()

print(end - start)

def parse_output(output):
    lines = output.split(' ')
    labeled_sentences = {}
    current_label = None
    for line in lines:
        if ':' in line:
            current_label, sentence = line.split(':', 1)
            labeled_sentences[current_label.strip()] = sentence.strip()
        else:
            if current_label:
                labeled_sentences[current_label] += ' ' + line.strip()
    return labeled_sentences

# Example output parsing
parsed_result = parse_output(result)

json_data = json.dumps(parsed_result, indent=4)
print(json_data)

Ho right hypochondrium discomfort since 5 6 months. Umbilical hernia detected in 20 December 2014. She is non diabetic and non hypertensive. She underwent LSCS 6 years ago. USG abdomen on 20 December 2014 revealed Umbilical hernia, benign hepatic cysts with pressure effect in RHD  fibroid. MDCT on 31 December 2014 showed hepatomegaly with asymmetrical fatty infiltration and multiple non enhancing cystic lesions in the right and caudate lobes showing regular peripheral rim without enhancing mural solid component. Suggest possibilities like biliary cyst adenomas  hepatic cysts. The largest lesion is compressing the right hepatic duct with prominence of corresponding biliary radicles. There are few prominent paraaortic and interaortocaval nodes. Gastric rugae are prominent. Visualized bowel loops show no significant wall thickening or dilatation. There is umbilical hernia. Patient again admitted on 03 January 2017 with history of right upper abdominal discomfort. No ho fever. She had past

In [11]:
# {
#     "ChiefComplaint": " Ho pain abdomen since 6 months",
#     "PresentProblemLocation": " Abdomen problem",
#     "PreProbDuration": " Complains from 6 months ago",
#     "PreProbOnset": " Gradually it happen",
#     "PreProbSeverity": " Moderate problem",
#     "AttackType": " No attack",
#     "SmokingHabitType": " Patient does not have smoking habit",
#     "PhysicalActivity": " Light Worker",
#     "ALCHOLIC": " Does not drink alcohol",
#     "DietryHabit": " Non Vegetarian",
#     "MealFrequency": " Prefer meals 2 3 times a day",
#     "VegCookingMedia": " Prefer oil mustrad oil",
#     "TreatmentType": " Treatment type CHOLECYSTECTOMY",
#     "SurgeryType": " Surgery type CHOLECYSTECTOMY",
#     "SurgeryDurationH": " nan",
#     "SurgeryDurationM": " nan",
#     "AnesthicTypeName": " Anesthic type General",
#     "AnesRecoverySmoothType": " Anes smooth recovery",
#     "RecorveryType": " NORMAL recovery",
#     "Followupdate": " Follow-up date 17 January 2015",
#     "DoctorRemark": " CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS\nHo pain abdomen since 6 months PresentProblem",
#     "AnesRecorveryType": " NORMAL recovery"
# }